## TYTAN tutorial おすすめ5（橋をかけろ）

最終更新：2023年6月25日 by ビネクラ安田

取り組み方

*   Google Colabで取り組む場合：ファイル＞ドライブにコピーを保存　<font color="red">※このファイルを直接編集しても保存されません</font>
*   Jupyter Notebookに移す場合：ファイル＞ダウンロード＞.ipynbをダウンロード

参考リンク１

*   [TYTANチュートリアル一覧](https://github.com/tytansdk/tytan_tutorial)
*   [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)

出展

*   [量子アニーリング（QUBO）で橋をかけろ（リンクブリッジ）を解く](https://vigne-cla.com/21-14/)

### 問題
橋をかけろ（リンクブリッジ）と呼ばれるパズル。

各数字から、数字の個数だけ上下左右に橋をかける。橋は最大で二重橋まで、他の数字の上をスルーすることはできない。また、全体が一つに繋がっている必要があり、複数の島に分離してしまうのはNG。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-14_1-300x300.png" width = 18%>
</div>

答えはこちら。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-14_3-300x300.png" width = 18%>
</div>

ブラウザ上で遊んでみると良い → [Puzzle Team](https://ja.puzzle-bridges.com/)

### このQUBO設定を使おう

<font color="red">「n個の量子ビットからm個を1にする」</font>

例）3個の量子ビットから2個を1にする
```
H = (q1 + q2 + q3 - 2)**2
```

例）この量子ビットを1にする（＝1個の量子ビットから1個を1にする）
```
H = (q1 - 1)**2
```

その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 制約条件

まず、橋がかかる可能性がある場所すべてに二重線を引き、それぞれを量子ビットに対応させる。今回の問題では16量子ビット使う。量子ビットが１になれば橋がかかったことになる。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-14_2-300x269.png" width = 23%>
</div>

あとは、数字に着目して**「可能性がある橋の中から数字の個数だけ1になる」**という設定を繰り返す。数字が９個あるので条件式も９式になる。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-14_4-768x291.png" width = 48%>
</div>

```
H += (q0 + q1 - 2)**2
H += (q0 + q1 + q2 + q3 - 3)**2
・・・
```

なお、今回の問題は簡単なので「全体が一つに繋がっていないといけない」という制約は特に設定しない。

### コード１

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [6]:
from tytan import *

#量子ビットを用意する
q00 = symbols('q00')
q01 = symbols('q01')
q02 = symbols('q02')
q03 = symbols('q03')
q04 = symbols('q04')
q05 = symbols('q05')
q06 = symbols('q06')
q07 = symbols('q07')
q08 = symbols('q08')
q09 = symbols('q09')
q10 = symbols('q10')
q11 = symbols('q11')
q12 = symbols('q12')
q13 = symbols('q13')
q14 = symbols('q14')
q15 = symbols('q15')

#各数字について「可能性がある橋の中から数字の個数だけ1になる」
H = 0
H += (q00 + q01 - 2)**2
H += (q00 + q01 + q02 + q03 - 3)**2
H += (q02 + q03 + q04 + q05 - 2)**2
H += (q04 + q05 + q06 + q07 - 2)**2
H += (q06 + q07 + q08 + q09 + q10 + q11 - 3)**2
H += (q08 + q09 - 1)**2
H += (q10 + q11 + q12 + q13 - 3)**2
H += (q12 + q13 + q14 + q15 - 3)**2
H += (q14 + q15 - 1)**2

#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo)

#上位5件
for r in result[:5]:
    print(r)

offset
50
[{'q00': 1, 'q01': 1, 'q02': 0, 'q03': 1, 'q04': 0, 'q05': 1, 'q06': 0, 'q07': 1, 'q08': 0, 'q09': 1, 'q10': 0, 'q11': 1, 'q12': 1, 'q13': 1, 'q14': 1, 'q15': 0}, -50.0, 2]
[{'q00': 1, 'q01': 1, 'q02': 1, 'q03': 0, 'q04': 0, 'q05': 1, 'q06': 0, 'q07': 1, 'q08': 0, 'q09': 1, 'q10': 0, 'q11': 1, 'q12': 1, 'q13': 1, 'q14': 0, 'q15': 1}, -50.0, 3]
[{'q00': 1, 'q01': 1, 'q02': 1, 'q03': 0, 'q04': 0, 'q05': 1, 'q06': 0, 'q07': 1, 'q08': 0, 'q09': 1, 'q10': 0, 'q11': 1, 'q12': 1, 'q13': 1, 'q14': 1, 'q15': 0}, -50.0, 2]
[{'q00': 1, 'q01': 1, 'q02': 1, 'q03': 0, 'q04': 0, 'q05': 1, 'q06': 0, 'q07': 1, 'q08': 0, 'q09': 1, 'q10': 1, 'q11': 0, 'q12': 1, 'q13': 1, 'q14': 0, 'q15': 1}, -50.0, 2]
[{'q00': 1, 'q01': 1, 'q02': 1, 'q03': 0, 'q04': 0, 'q05': 1, 'q06': 0, 'q07': 1, 'q08': 1, 'q09': 0, 'q10': 0, 'q11': 1, 'q12': 1, 'q13': 1, 'q14': 0, 'q15': 1}, -50.0, 2]


### コストの追加

さて、エネルギー=-50の解がたくさん出てしまったが実質１パターンだけである。1本橋は[1, 0]でも[0, 1]でも表現できるため、解がちょっとした組合せ爆発を起こしている。

そこで解を一つに絞るための**弱い条件設定**を追加する。偶数ビットを奇数ビットに対して少しだけ1になりやすくする（係数0.1をかけること）。これにより[0, 1]より[1, 0]が優先される。

### コード２

In [8]:
from tytan import symbols, Compile, sampler

#量子ビットを用意する
q00 = symbols('q00')
q01 = symbols('q01')
q02 = symbols('q02')
q03 = symbols('q03')
q04 = symbols('q04')
q05 = symbols('q05')
q06 = symbols('q06')
q07 = symbols('q07')
q08 = symbols('q08')
q09 = symbols('q09')
q10 = symbols('q10')
q11 = symbols('q11')
q12 = symbols('q12')
q13 = symbols('q13')
q14 = symbols('q14')
q15 = symbols('q15')

#各数字について「可能性がある橋の中から数字の個数だけ1になる」
H = 0
H += (q00 + q01 - 2)**2
H += (q00 + q01 + q02 + q03 - 3)**2
H += (q02 + q03 + q04 + q05 - 2)**2
H += (q04 + q05 + q06 + q07 - 2)**2
H += (q06 + q07 + q08 + q09 + q10 + q11 - 3)**2
H += (q08 + q09 - 1)**2
H += (q10 + q11 + q12 + q13 - 3)**2
H += (q12 + q13 + q14 + q15 - 3)**2
H += (q14 + q15 - 1)**2

#（弱い条件）偶数ビットの方がほんの少しだけ1になりやすくする（＝[0, 1]より[1, 0]を優先する）
H += 0.1 * (q00 - 1)**2
H += 0.1 * (q02 - 1)**2
H += 0.1 * (q04 - 1)**2
H += 0.1 * (q06 - 1)**2
H += 0.1 * (q08 - 1)**2
H += 0.1 * (q10 - 1)**2
H += 0.1 * (q12 - 1)**2
H += 0.1 * (q14 - 1)**2

#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo)

#上位5件
for r in result[:5]:
    print(r)

offset
50.8000000000000
[{'q00': 1, 'q01': 1, 'q02': 1, 'q03': 0, 'q04': 1, 'q05': 0, 'q06': 1, 'q07': 0, 'q08': 1, 'q09': 0, 'q10': 1, 'q11': 0, 'q12': 1, 'q13': 1, 'q14': 1, 'q15': 0}, -50.800000000000004, 99]
[{'q00': 1, 'q01': 1, 'q02': 1, 'q03': 0, 'q04': 1, 'q05': 0, 'q06': 1, 'q07': 0, 'q08': 1, 'q09': 0, 'q10': 1, 'q11': 1, 'q12': 1, 'q13': 0, 'q14': 1, 'q15': 1}, -48.800000000000004, 1]


これで解が１パターンに絞られた。橋をかけると模範解答の通りである。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-14_3-300x300.png" width = 18%>
</div>

なお、弱い条件は「偶数ビットを奇数ビットに対して少しだけ1になりやすくする」に限らず、温度計パズルで用いたような降順設定でも実現できる。興味があれば試してみてほしい。